In [1]:
pip install pyttsx3

In [2]:
pip install pywin32

In [3]:
pip install numpy

In [4]:
pip install opencv-python

# Exercise 1.2

<p style="font-size:15px;">To count the number of cars going to the city centre, I have implemented something called a box-point-collision. The way the collision algorithm works, is that for a car's top left corner coordinates, if those coordinates "collide" or fall within the box, it will count as a collision and increase the counter by 1. The placement of the box is also very important as it has to be at an area where no other cars would be able to invoke the algorithm.</p>

<p style="font-size:14px; color:red;">The function will contain comments to explain the code more in-depth.</p>

In [5]:
import cv2

In [6]:
def collision(x, y, bx1, by1, bx2, by2):
    # check if the point lies between the box's coordinates, if yes, return True, if no, return False
    if x >= bx1 and x <= bx2 and y >= by1 and y <= by2:
        return True
    else:
        return False

In [7]:
def object_detection(file):
    # collision box coordinates
    box_x1 = 561
    box_x2 = 567
    box_y1 = 101
    box_y2 = 139
    # global counter
    counter = 0
    # create VideoCapture object and read frames from input file
    video = cv2.VideoCapture(file)
    # check if video opened successfully
    if (video.isOpened()== False):
        print("Error opening video file.")
    else:
        print("Video opened successfully!")
    # global baseline
    initial_frame = None
    # loop until video completes
    while True:
        # capture frame-by-frame
        check, frame = video.read()
        # if there are no more frames to read, exit
        if frame is None:
            break
        # option to exit with 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
            
        # [600, 1040] is the height and width of the video
        # we are only interested in the main road which is bound to [265:, 0:]
        main_road = frame[265:600, 0:1040]
        # draw a line to separate the video and main road
        cv2.line(frame, (0, 260), (1040, 260), (0, 0, 255), 5, 8, 0)
        
        # gray conversion
        gray_frame = cv2.cvtColor(main_road, cv2.COLOR_BGR2GRAY)
        # set first captured frame as the baseline
        if initial_frame is None:
            initial_frame = gray_frame
        # find the difference of the current frame from the baseline 
        delta_frame = cv2.absdiff(initial_frame, gray_frame)
        # if a pixel value is more than 40, assign it to 255
        _, threshold_frame = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)
        # noise reduction
        blur_frame = cv2.GaussianBlur(threshold_frame, (5, 5), 0)
        # this is the "collision box" where we check if cars "collide with"
        cv2.rectangle(main_road, (box_x1, box_y1), (box_x2, box_y2), (0, 255, 255), 1)
        
        # identify all the contours in the video
        (contours, _) = cv2.findContours(blur_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for c in contours:
            # if the contour area is more than a certain value, draw a rectangle around it
            if cv2.contourArea(c) > 3000:
                x, y, w, h, = cv2.boundingRect(c)
                cv2.rectangle(main_road, (x, y), (x + w, y + h), (0, 255, 0), 1)
                # draw a tracking point of top left corner of bounding box
                cv2.circle(main_road, (x, y), 1, (0, 0, 255), 2)
                # using the tracking point coordinates, check if they collide with the box
                # if True, increment counter by 1
                hit = collision(x, y, box_x1, box_y1, box_x2, box_y2)
                if hit:
                    counter += 1
        
        # write counter
        cv2.putText(frame, ("Number of cars going to the city centre: " + str(counter)), (10, 240), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        # show video
        cv2.imshow("CCTV", frame)
        
    video.release()
    cv2.destroyAllWindows()
    print("Number of cars going to the city centre:", counter)

In [8]:
object_detection('Exercise1_Files/Traffic_Laramie_1.mp4')

Video opened successfully!
Number of cars going to the city centre: 6


In [9]:
object_detection('Exercise1_Files/Traffic_Laramie_2.mp4')

Video opened successfully!
Number of cars going to the city centre: 4


## References
<ul>
    <li>
        <a href=https://www.coursera.org/learn/uol-cm3065-intelligent-signal-processing/ungradedLab/0PXRU/7-104-exercise-15-introduction-to-motion-detection-with-opencv-and-python/lab?path=%2Fnotebooks%2FExercises%2FExercise%252015.%2520Introduction%2520to%2520motion%2520detection%2520with%2520OpenCV%2520and%2520Python.ipynb>Coursera Lab</a>
    </li>
</ul>